### Libraries

In [1]:
import pandas as pd
import numpy  as np
from backtesting import Strategy
from backtesting import Backtest
from datetime import datetime

C:\Users\MINK\AppData\Roaming\Python\Python39\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### OHLC data and EDA

In [2]:
# read in csv data
df = pd.read_csv("EURUSD_Daily_201201020000_202112310000.csv",sep="\t")
# create 'date' columns as datetime obj
df['date'] = pd.to_datetime(df['<DATE>'])
# drop unwanted columns
df.drop(columns=['<DATE>','<TICKVOL>','<VOL>', '<SPREAD>'], inplace=True)
# rename columns
df.columns = ['open','high','low','close','date']
# set date as index 
df.set_index('date', inplace=True)
df.shape

(2661, 4)

### Support and Resistance finding functions

In [3]:
def support(df1, l, n1, n2): 
    """
    support level finder function
    l: observing candle
    n1: candle before l
    n2: candle after l
    
    """
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2):
    """
    resistance level finder function
    l: observing candle
    n1: candle before l
    n2: candle after l
    
    """
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

### Candle sticks pattern search

In [4]:
length = len(df)
high   = list(df['high'])
low    = list(df['low'])
close  = list(df['close'])
open   = list(df['open'])

bodydiff = [0] * length
highdiff = [0] * length
lowdiff  = [0] * length
ratio1   = [0] * length
ratio2   = [0] * length

def isEngulfing(l):
    """
    detect Engulfing candle stick pattern
    where: 0 is no-trend 
           1 is bearish
           2 is bullish
    """
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.0002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): 
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):
        return 2
    else:
        return 0
       
def isStar(l):
    """
    detect Star candle stick pattern
    where: 0 is no-trend 
           1 is bearish
           2 is bullish    
    """
    bodydiffmin = 0.0002
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    """
    check if the current price is close to 
    Resistance level
    """
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    """
    check if the current price is close to 
    Support level
    """
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

### Signals

In [5]:
n1=2 # 2 candles before l candle
n2=2 # 2 candles after l candle
backCandles=45 # detect signal in current 45 candle stickcs
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):
        signal[row] = 1 # sell
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):
        signal[row] = 2 # buy
    else:
        signal[row] = 0 # do nothing
        
df['signal']=signal

In [6]:
df.columns = ['Open', 'High', 'Low', 'Close','signal']
print(df['signal'].value_counts())
df.head()

0    2471
1     115
2      75
Name: signal, dtype: int64


,Open,High,Low,Close,signal
date,,,,,
2012-01-02,1.29443,1.29681,1.29168,1.29338,0
2012-01-03,1.29337,1.30762,1.29316,1.30509,0
2012-01-04,1.30509,1.30719,1.28974,1.29361,0
2012-01-05,1.29359,1.29434,1.27702,1.27904,0
2012-01-06,1.27907,1.28121,1.26972,1.27188,0


In [7]:
def SIGNAL():
    return df.signal

class MyCandlesStrat(Strategy):
    initsize = 0.1
    mysize = initsize
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        stop= 600e-4 # 600 pips
        take= 600e-4 # 600 pips
        super().next()
        if self.signal1==2:
            sl1 = self.data.Close[-1] - stop
            tp1 = self.data.Close[-1] + take
            self.buy(sl=sl1,tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + take
            tp1 = self.data.Close[-1] - stop
            self.sell(sl=sl1,tp=tp1)

In [8]:
bt = Backtest(df, MyCandlesStrat, cash=100, commission=.002)
stat = bt.run()
stat

Start                     2012-01-02 00:00:00
End                       2021-12-31 00:00:00
Duration                   3651 days 00:00:00
Exposure Time [%]                   84.441939
Equity Final [$]                   147.018044
Equity Peak [$]                     149.43845
Return [%]                          47.018044
Buy & Hold Return [%]              -12.113996
Return (Ann.) [%]                    3.717062
Volatility (Ann.) [%]                6.758855
Sharpe Ratio                         0.549954
Sortino Ratio                        0.840124
Calmar Ratio                         0.279144
Max. Drawdown [%]                  -13.315925
Avg. Drawdown [%]                   -1.301616
Max. Drawdown Duration     1355 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   42
Win Rate [%]                        83.333333
Best Trade [%]                       5.438496
Worst Trade [%]                      -5.72629
Avg. Trade [%]                    

In [10]:
bt.plot()

Row(id='2132', ...)